In [115]:
!sudo apt-get update -y
!sudo apt-get install python3.9


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [88]:
!pip install tensor-dash

In [89]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
dataset = pd.read_csv('/content/drive/MyDrive/hackathon/plant/train.csv')

In [93]:
!pip install tensorflow
import tensorflow as tf
from tensorflow import keras


datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        zca_whitening=False,
        rotation_range=180,
        zoom_range = 0.15,
        width_shift_range=0.15,
        height_shift_range=0.15,
        horizontal_flip=True,
        vertical_flip=True)

In [95]:
!pip install scikit-learn
import pandas as pd
from sklearn.model_selection import train_test_split
X_train, X_valid = train_test_split(dataset, test_size=0.05, shuffle=False)

In [96]:
print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_valid.shape}")


Training set shape: (1729, 5)
Validation set shape: (92, 5)


In [97]:
print(X_train['image_id'].head())
print(X_valid['image_id'].head())


0    Train_0
1    Train_1
2    Train_2
3    Train_3
4    Train_4
Name: image_id, dtype: object
1729    Train_1729
1730    Train_1730
1731    Train_1731
1732    Train_1732
1733    Train_1733
Name: image_id, dtype: object


In [98]:
X_train['image_id'] = X_train['image_id'].apply(lambda x: os.path.join('/content/drive/MyDrive/hackathon/plant/images', x + '.jpg'))
X_valid['image_id'] = X_valid['image_id'].apply(lambda x: os.path.join('/content/drive/MyDrive/hackathon/plant/images', x + '.jpg'))

X_train['image_id'] = X_train['image_id'].str.strip()
X_valid['image_id'] = X_valid['image_id'].str.strip()


In [99]:
for path in X_train['image_id'][:5]:
    if not os.path.exists(path):
        print(f"File not found: {path}")
for path in X_valid['image_id'][:5]:
    if not os.path.exists(path):
        print(f"File not found: {path}")


In [100]:

BATCH_SIZE = 8
train_generator = datagen.flow_from_dataframe(
    X_train,
    directory='/content/drive/MyDrive/hackathon/plant/images',
    x_col='image_id',
    y_col=['healthy', 'multiple_diseases', 'rust', 'scab'],
    target_size=(512, 512),
    class_mode='raw',
    batch_size=BATCH_SIZE,
    shuffle=True
)

valid_generator = datagen.flow_from_dataframe(
    X_valid,
    directory='/content/drive/MyDrive/hackathon/plant/images',
    x_col='image_id',
    y_col=['healthy', 'multiple_diseases', 'rust', 'scab'],
    target_size=(512, 512),
    class_mode='raw',
    batch_size=BATCH_SIZE,
    shuffle=False
)

print(f"Number of training samples: {train_generator.samples}")
print(f"Number of validation samples: {valid_generator.samples}")


Found 1729 validated image filenames.
Found 92 validated image filenames.
Number of training samples: 1729
Number of validation samples: 92


In [102]:
import tensorflow as tf
xception_model = tf.keras.models.Sequential([
  tf.keras.applications.xception.Xception(include_top=False, weights='imagenet', input_shape=(512, 512, 3)),
   tf.keras.layers.GlobalAveragePooling2D(),
   tf.keras.layers.Dense(4,activation='softmax')
])
xception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
xception_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ xception (Functional)                │ (None, 16, 16, 2048)        │      20,861,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_5           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 4)                   │           8,196 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,869,676 (79.61 MB)

 Trainable params: 20,815,148 (79.40 MB)

 Non-trainable params: 54,528 (213.00 KB)

In [103]:
densenet_model = tf.keras.models.Sequential([
    tf.keras.applications.densenet.DenseNet121(include_top=False, weights='imagenet',input_shape=(512, 512, 3)),
   tf.keras.layers.GlobalAveragePooling2D(),
   tf.keras.layers.Dense(4,activation='softmax')
])
densenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
densenet_model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)             │ (None, 16, 16, 1024)        │       7,037,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_6           │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 4)                   │           4,100 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,041,604 (26.86 MB)

 Trainable params: 6,957,956 (26.54 MB)

 Non-trainable params: 83,648 (326.75 KB)

In [104]:
inputs = tf.keras.Input(shape=(512, 512, 3))

xception_output = xception_model(inputs)
densenet_output = densenet_model(inputs)

outputs = tf.keras.layers.average([densenet_output, xception_output])


model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20            │ (None, 512, 512, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_6 (Sequential) │ (None, 4)              │      7,041,604 │ input_layer_20[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_5 (Sequential) │ (None, 4)              │     20,869,676 │ input_layer_20[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ average_6 (Average)       │ (None, 4)              │              0 │ sequential_6[0][0],    │
│                           │                        │                │ sequential_5[0][0]     │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 27,911,280 (106.47 MB)

 Trainable params: 27,773,104 (105.95 MB)

 Non-trainable params: 138,176 (539.75 KB)

In [106]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [107]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('model.keras', monitor='val_accuracy', verbose=2, save_best_only=True)


In [110]:
print(train_generator)
print(valid_generator)


In [111]:
x, y = next(train_generator)
print(f"Inputs shape: {x.shape}, Labels shape: {y.shape}")


Inputs shape: (8, 512, 512, 3), Labels shape: (8, 4)


In [112]:
for i in range(5):
        try:
            x, y = next(train_generator)
            print(f"Batch {i}: Inputs shape: {x.shape}, Labels shape: {y.shape}")
        except StopIteration:
            print(f"train_generator exhausted after {i} batches.")
            break

for i in range(5):
        try:
            x, y = next(valid_generator)
            print(f"Batch {i}: Inputs shape: {x.shape}, Labels shape: {y.shape}")
        except StopIteration:
            print(f"valid_generator exhausted after {i} batches.")
            break

Batch 0: Inputs shape: (8, 512, 512, 3), Labels shape: (8, 4)
Batch 1: Inputs shape: (8, 512, 512, 3), Labels shape: (8, 4)
Batch 2: Inputs shape: (8, 512, 512, 3), Labels shape: (8, 4)
Batch 3: Inputs shape: (8, 512, 512, 3), Labels shape: (8, 4)
Batch 4: Inputs shape: (8, 512, 512, 3), Labels shape: (8, 4)
Batch 0: Inputs shape: (8, 512, 512, 3), Labels shape: (8, 4)
Batch 1: Inputs shape: (8, 512, 512, 3), Labels shape: (8, 4)
Batch 2: Inputs shape: (8, 512, 512, 3), Labels shape: (8, 4)
Batch 3: Inputs shape: (8, 512, 512, 3), Labels shape: (8, 4)
Batch 4: Inputs shape: (8, 512, 512, 3), Labels shape: (8, 4)


In [118]:
import tensorflow as tf
print(tf.__version__)

2.17.1


In [119]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 793.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 81.5 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [126]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set up early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Set up model checkpoint to save the best model based on validation loss
model_checkpoint = ModelCheckpoint('model.keras', save_best_only=True, monitor='val_loss')

# Train the model using the generators
history = model.fit(
    train_generator,
    epochs=EPOCHS,  # Keep using your existing EPOCHS variable
    validation_data=valid_generator,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/100


AttributeError: 'NoneType' object has no attribute 'items'

In [125]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set up early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Set up model checkpoint to save the best model based on validation loss
model_checkpoint = ModelCheckpoint('model.keras', save_best_only=True, monitor='val_loss')

# Train the model using the generators
history = model.fit(
    train_generator,
    epochs=10,  # Adjust epochs based on your needs
    validation_data=valid_generator,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/10


AttributeError: 'NoneType' object has no attribute 'items'

In [124]:
model_history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=valid_generator,
    callbacks=[model_checkpoint, lr_callback]
)


Epoch 1/100


AttributeError: 'NoneType' object has no attribute 'items'